In [674]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer ## to creae bag of words

from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
import math



pd.set_option('display.max_columns', 100)
pd.set_option('display.expand_frame_repr', True)
pd.options.display.max_colwidth = 100000

import warnings
warnings.filterwarnings("ignore")

In [675]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [711]:
talents = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/potential-talents - Aspiring human resources - seeking human resources.csv")
talents

,id,job_title,location,connection,fit
0,1,2019 C.T. Bauer College of Business Graduate (Magna Cum Laude) and aspiring Human Resources professional,"Houston, Texas",85,NaN
1,2,Native English Teacher at EPIK (English Program in Korea),Kanada,500+,NaN
2,3,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,NaN
3,4,People Development Coordinator at Ryan,"Denton, Texas",500+,NaN
4,5,Advisory Board Member at Celal Bayar University,"İzmir, Türkiye",500+,NaN
...,...,...,...,...,...
99,100,Aspiring Human Resources Manager | Graduating May 2020 | Seeking an Entry-Level Human Resources Position in St. Louis,"Cape Girardeau, Missouri",103,NaN
100,101,Human Resources Generalist at Loparex,"Raleigh-Durham, North Carolina Area",500+,NaN
101,102,Business Intelligence and Analytics at Travelers,Greater New York City Area,49,NaN
102,103,Always set them up for Success,Greater Los Angeles Area,500+,NaN


In [712]:
## create new column to label selected employees and save it to new dtabse that contain ids 
talents['selected'] = 0
## create new column to store the final similarity calculated by the mosel
talents['similarity']= 0


In [713]:
talents2 = talents.copy()

In [714]:
## clean the text and convert it to lower case

def clean_text(job_titles):
    ps = PorterStemmer()
    employee_titles = []
    for i in range(0, len(job_titles)):
        titles = re.sub('[^a-zA-Z]', ' ', job_titles[i])
        titles = titles.lower()
        titles = titles.split()
        titles = [word for word in titles if not word in stopwords.words('english')]
        titles = ' '.join(titles)
        employee_titles.append(titles)
    return employee_titles

In [681]:
##  user to input the job title
key1_titel = input('write the jobe describtion or skils you are loking for: ')
if key1_titel:
    key1_titel = [key1_titel]
else:
    key1_titel = []

write the jobe describtion or skils you are loking for: Aspiring Human resource


In [731]:
##  user to input the job location
key1_location = input('best location: ')
if key1_location:
    key1_location = [key1_location]
else:
    key1_location = []

best location: 


In [732]:


## define sigmoid function to calculate the probability from similarity
def sigmoid(x):
    return 1 / (1 + math.exp(-x))

In [733]:
## OpenAI Tranformer language model
model = SentenceTransformer('all-MiniLM-L6-v2')


# lets try to make system work as folowing looking for location similarity and job similarity individualy then combine the results. after that the model will take the selected employee effect into consideration


In [734]:
job_title = talents['job_title']
location = talents['location']

In [735]:
# clean the text
job_title = clean_text(job_title)
location = clean_text(location)


In [736]:
# Embedding the titles
titles_embeddings = model.encode(job_title, show_progress_bar=True)

# Printing the embeddings
titles_embeddings.shape

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

(104, 384)

In [737]:
# Embedding the location
location_embeddings = model.encode(location)

# Printing the embeddings
location_embeddings.shape

(104, 384)

In [738]:
# Embedding the titles keys
if key1_titel:
  key_title_embeddings = model.encode(key1_titel)

# Printing the embeddings
  key_title_embeddings.shape

In [739]:
# Embedding the locations keys
if key1_location:
  key_location_embeddings = model.encode(key1_location)

# Printing the embeddings
  key_location_embeddings.shape

In [740]:
## lets find the similarities between keys and values, then between employess and the selected one if there is a selected employee
# title similarity 
title_similairt = cosine_similarity(titles_embeddings, key_title_embeddings)
## loation similarity
location_similairt = cosine_similarity(location_embeddings, key_location_embeddings)


In [742]:
## all cases to calculate the similarity
def similarity_calculate(employee_num=employee_number, key1_location=key1_location, key1_titel=key1_titel):
    global  employee_number
    if not employee_number:
        if not key1_location:
            if talents['similarity'].sum() == 0:
                  talents['similarity'] = pd.Series(title_similairt.ravel())
            else:
                  talents['similarity'] = (talents['similarity'] + pd.Series(title_similairt.ravel()))/2
        elif not key1_titel:
            if talents['similarity'].sum() == 0:
                  talents['similarity'] = pd.Series(location_similairt.ravel())
            else:
                  talents['similarity'] =  (talents['similarity'] + pd.Series(location_similairt.ravel()))/2
        else:  
              combin_sim = (location_similairt + title_similairt)/2
            if talents['similarity'].sum() == 0:
                  talents['similarity'] = combin_sim
            else:
                    talents['similarity'] = (talents['similarity'] + pd.Series(combin_sim.ravel()))/2
      ## lets combine the similarity by averaging the values
      talents['fit'] = [sigmoid(sim) for sim in talents['similarity']]

    return talents.sort_values(by=['fit'], ascending=False).head(15)
      
          
          
    else:
        talents.loc[talents['id'] == int(employee_number), 'selected'] = 1
        ## lets extract the index 
        idx= talents[talents['id'] == int(employee_number)].index[0]
        selecte_title = titles_embeddings[idx]
        selected_location = location_embeddings[idx]

        select_emp_title_sim = cosine_similarity(titles_embeddings, selecte_title.reshape(1,-1))
        select_emp_location_sim = cosine_similarity(location_embeddings, selected_location.reshape(1,-1))

        ### lets average with previous state
        combin_emp_sim = (select_emp_title_sim + select_emp_location_sim)/2
        talents['similarity'] = (0.4*talents['similarity'] +  0.6*pd.Series(combin_emp_sim.ravel()))/2
        ## lets combine the similarity by averaging the values
        talents['fit'] = [sigmoid(sim) for sim in talents['similarity']]

        employee_number = ''

    return talents.sort_values(by=['fit'], ascending=False).head(16)

In [727]:
similarity_calculate()

,id,job_title,location,connection,fit,selected,similarity
45,46,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,0.718813,0,0.938582
16,17,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,0.718813,0,0.938582
2,3,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,0.718813,0,0.938582
32,33,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,0.718813,0,0.938582
57,58,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,0.718813,0,0.938582
96,97,Aspiring Human Resources Professional,"Kokomo, Indiana Area",71,0.718813,0,0.938582
20,21,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,0.718813,0,0.938582
59,60,Aspiring Human Resources Specialist,Greater New York City Area,1,0.712289,0,0.906527
23,24,Aspiring Human Resources Specialist,Greater New York City Area,1,0.712289,0,0.906527
35,36,Aspiring Human Resources Specialist,Greater New York City Area,1,0.712289,0,0.906527


In [728]:
## select an employe
employee_number = input('add employee number to be selectd as top performer: ')


add employee number to be selectd as top performer: 24


In [729]:
similarity_calculate()

,id,job_title,location,connection,fit,selected,similarity
23,24,Aspiring Human Resources Specialist,Greater New York City Area,1,0.618056,1,0.481305
59,60,Aspiring Human Resources Specialist,Greater New York City Area,1,0.618056,0,0.481305
48,49,Aspiring Human Resources Specialist,Greater New York City Area,1,0.618056,0,0.481305
5,6,Aspiring Human Resources Specialist,Greater New York City Area,1,0.618056,0,0.481305
35,36,Aspiring Human Resources Specialist,Greater New York City Area,1,0.618056,0,0.481305
20,21,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,0.595214,0,0.385564
32,33,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,0.595214,0,0.385564
16,17,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,0.595214,0,0.385564
57,58,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,0.595214,0,0.385564
45,46,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,0.595214,0,0.385564


In [730]:
##  user to input the job title
key1_titel = input('write the jobe describtion or skils you are loking for: ')
if key1_titel:
    key1_titel = [key1_titel]
else:
    key1_titel = []

write the jobe describtion or skils you are loking for: Internship


In [743]:
similarity_calculate()

,id,job_title,location,connection,fit,selected,similarity
72,73,"Aspiring Human Resources Manager, seeking internship in Human Resources.","Houston, Texas Area",7,0.616916,0,0.476481
26,27,Aspiring Human Resources Management student seeking an internship,"Houston, Texas Area",500+,0.615975,0,0.472501
28,29,Aspiring Human Resources Management student seeking an internship,"Houston, Texas Area",500+,0.615975,0,0.472501
48,49,Aspiring Human Resources Specialist,Greater New York City Area,1,0.597211,0,0.393857
23,24,Aspiring Human Resources Specialist,Greater New York City Area,1,0.597211,1,0.393857
5,6,Aspiring Human Resources Specialist,Greater New York City Area,1,0.597211,0,0.393857
35,36,Aspiring Human Resources Specialist,Greater New York City Area,1,0.597211,0,0.393857
59,60,Aspiring Human Resources Specialist,Greater New York City Area,1,0.597211,0,0.393857
45,46,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,0.582820,0,0.334362
16,17,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,0.582820,0,0.334362


In [744]:
## select an employe
employee_number = input('add employee number to be selectd as top performer: ')


add employee number to be selectd as top performer: 29


In [745]:
similarity_calculate()

,id,job_title,location,connection,fit,selected,similarity
28,29,Aspiring Human Resources Management student seeking an internship,"Houston, Texas Area",500+,0.597366,1,0.394500
26,27,Aspiring Human Resources Management student seeking an internship,"Houston, Texas Area",500+,0.597366,0,0.394500
72,73,"Aspiring Human Resources Manager, seeking internship in Human Resources.","Houston, Texas Area",7,0.595933,0,0.388547
0,1,2019 C.T. Bauer College of Business Graduate (Magna Cum Laude) and aspiring Human Resources professional,"Houston, Texas",85,0.571669,0,0.288663
56,57,2019 C.T. Bauer College of Business Graduate (Magna Cum Laude) and aspiring Human Resources professional,"Houston, Texas",85,0.571669,0,0.288663
30,31,2019 C.T. Bauer College of Business Graduate (Magna Cum Laude) and aspiring Human Resources professional,"Houston, Texas",85,0.571669,0,0.288663
18,19,2019 C.T. Bauer College of Business Graduate (Magna Cum Laude) and aspiring Human Resources professional,"Houston, Texas",85,0.571669,0,0.288663
14,15,2019 C.T. Bauer College of Business Graduate (Magna Cum Laude) and aspiring Human Resources professional,"Houston, Texas",85,0.571669,0,0.288663
13,14,2019 C.T. Bauer College of Business Graduate (Magna Cum Laude) and aspiring Human Resources professional,"Houston, Texas",85,0.571669,0,0.288663
43,44,2019 C.T. Bauer College of Business Graduate (Magna Cum Laude) and aspiring Human Resources professional,"Houston, Texas",85,0.571669,0,0.288663
